#### Creating the Training Dataset

In [1]:
import tensorflow as tf

shakespeare_url = "https://homl.info/shakespeare"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)

with open(filepath) as f:
    shakespeare_text = f.read()

1115394/1115394 [==============================] - 2s 1us/step


In [18]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [19]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In [20]:
encoded -= 2 # drop tokens 0 (pad) and 1 (unknown), which we will not use
n_tokens = text_vec_layer.vocabulary_size()-2
n_tokens

39

In [21]:
dataset_size = len(encoded)
dataset_size

1115394

In [22]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
        
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [23]:
list(to_dataset(text_vec_layer(["To be"])[0], length=4))

[(<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[ 4,  5,  2, 23]], dtype=int64)>,
  <tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[ 5,  2, 23,  3]], dtype=int64)>)]

In [24]:
length = 100 # window length
tf.random.set_seed(42)

train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

#### Building and Training the Char-RNN Model

In [25]:
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [26]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint("my_shakespeare_model.h5", monitor="val_accuracy", save_best_only=True)
history = model.fit(train_set, validation_data=valid_set, epochs=10, callbacks=[model_ckpt])

Epoch 1/10
31247/31247 [==============================] - 2391s 76ms/step - loss: 1.3926 - accuracy: 0.5744 - val_loss: 1.6078 - val_accuracy: 0.5357
Epoch 2/10


C:\Users\User\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31247/31247 [==============================] - 2991s 95ms/step - loss: 1.2935 - accuracy: 0.5974 - val_loss: 1.5845 - val_accuracy: 0.5396
Epoch 3/10
31247/31247 [==============================] - 2924s 93ms/step - loss: 1.2747 - accuracy: 0.6015 - val_loss: 1.5732 - val_accuracy: 0.5440
Epoch 4/10
31247/31247 [==============================] - 2891s 92ms/step - loss: 1.2657 - accuracy: 0.6036 - val_loss: 1.5677 - val_accuracy: 0.5448
Epoch 5/10
31247/31247 [==============================] - 2896s 92ms/step - loss: 1.2588 - accuracy: 0.6051 - val_loss: 1.5621 - val_accuracy: 0.5444
Epoch 6/10
31247/31247 [==============================] - 2902s 92ms/step - loss: 1.2550 - accuracy: 0.6059 - val_loss: 1.5571 - val_accuracy: 0.5470
Epoch 7/10
31247/31247 [==============================] - 2863s 91ms/step - loss: 1.2510 - accuracy: 0.6067 - val_loss: 1.5485 - val_accuracy: 0.5495
Epoch 8/10
31247/31247 [==============================] - 2857s 91ms/step - loss: 1.2481 - accuracy: 0.6074 - v

This model does not hnadle text preprocessing, let's wrap it in a final a model containing `tf.keras.layers.TextVectorization` layer as the first layer and a `tf.keras.layers.Lambda` layer to subtract 2 from the character IDs since we are not using  the padding and unknown tokens for now.

In [32]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X:X-2), # no <PAD> or <UNKNOWN> tokens
    model
])

In [33]:
# Predicting the Next Character
y_proba = shakespeare_model.predict(["To be or not to b"])[0,-1]
y_pred = tf.argmax(y_proba)

text_vec_layer.get_vocabulary()[y_pred+2]

1/1 [==============================] - 1s 574ms/step


'e'

#### Generating Fake Shakespearean Text

In [34]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 1, 0, 2, 1, 0, 0, 1]], dtype=int64)>

In [36]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba)/temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
    return text_vec_layer.get_vocabulary()[char_id+2]

In [40]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [41]:
tf.random.set_seed(42)
print(extend_text("To be or not to be", temperature=0.01))

1/1 [==============================] - 0s 53ms/step
To be or not to be a shame.

provost:
i will be so the duke of sorro


In [42]:
tf.random.set_seed(42)
print(extend_text("To be or not to be", temperature=1))

1/1 [==============================] - 0s 60ms/step
To be or not to be noble an altly,
every profession in nothing; most


In [43]:
tf.random.set_seed(42)
print(extend_text("To be or not to be", temperature=100))

1/1 [==============================] - 0s 71ms/step
To be or not to belfzzy.cik pua!&j. :bhfr3'db
t,bzhbp.rkp,qzz!$ badp


#### Stateful RNN

In [46]:
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length + 1)).batch(1)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [47]:
stateful_train_set = to_dataset_for_stateful_rnn(encoded[:1_000_000], length=length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000], length=length)
stateful_test_set = to_dataset_for_stateful_rnn(encoded[1_060_000:], length=length)

In [53]:
stateful_rnn_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16, batch_input_shape=[1, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [63]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [64]:
stateful_rnn_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = stateful_rnn_model.fit(stateful_train_set, validation_data=stateful_valid_set,
                    epochs=10, callbacks=[ResetStatesCallback(), model_ckpt])

Epoch 1/10
9999/9999 [==============================] - 376s 37ms/step - loss: 1.8725 - accuracy: 0.4493 - val_loss: 1.7127 - val_accuracy: 0.4866
Epoch 2/10
9999/9999 [==============================] - 374s 37ms/step - loss: 1.5643 - accuracy: 0.5280 - val_loss: 1.6243 - val_accuracy: 0.5138
Epoch 3/10
9999/9999 [==============================] - 371s 37ms/step - loss: 1.4870 - accuracy: 0.5477 - val_loss: 1.5849 - val_accuracy: 0.5262
Epoch 4/10
9999/9999 [==============================] - 367s 37ms/step - loss: 1.4479 - accuracy: 0.5578 - val_loss: 1.5609 - val_accuracy: 0.5323
Epoch 5/10
9999/9999 [==============================] - 363s 36ms/step - loss: 1.4235 - accuracy: 0.5641 - val_loss: 1.5439 - val_accuracy: 0.5366
Epoch 6/10
9999/9999 [==============================] - 364s 36ms/step - loss: 1.4069 - accuracy: 0.5684 - val_loss: 1.5359 - val_accuracy: 0.5383
Epoch 7/10
9999/9999 [==============================] - 365s 36ms/step - loss: 1.3950 - accuracy: 0.5714 - val_loss: 1

### Converting the stateful RNN to a stateless RNN and using it

In [65]:
stateless_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [ ]:
stateless_model.build(tf.TensorShape([None, None]))
stateless_model.set_weights(st.get_weights())